# A Tiny Temperature Converter Library
## Written in Julia (a naive approach)

Based on a wonderful example from _Erik Engheim_

We start with some type definitions!

In [1]:
abstract type Temperature end

struct Celsius <: Temperature
    value::Float64
end

struct Kelvin <: Temperature
   value::Float64 
end

struct Fahrenheit <: Temperature
    value::Float64
end

Define the promotion rules, so Julia knows our preferences when mixing types.

In [2]:
import Base: promote_rule  # we import the `promote_rule` function to add our own methods

promote_rule(::Type{Kelvin}, ::Type{Celsius})     = Kelvin
promote_rule(::Type{Fahrenheit}, ::Type{Celsius}) = Celsius
promote_rule(::Type{Fahrenheit}, ::Type{Kelvin})  = Kelvin

promote_rule (generic function with 139 methods)

We implement the conversion logic by adding methods to the `convert` function, available in the `Base` of Julia.

In [3]:
import Base: convert  # again, we add our own methods to `convert`

convert(::Type{Kelvin},  t::Celsius)     = Kelvin(t.value + 273.15)
convert(::Type{Kelvin},  t::Fahrenheit)  = Kelvin(Celsius(t))
convert(::Type{Celsius}, t::Kelvin)      = Celsius(t.value - 273.15)
convert(::Type{Celsius}, t::Fahrenheit)  = Celsius((t.value - 32)*5/9)
convert(::Type{Fahrenheit}, t::Celsius)  = Fahrenheit(t.value*9/5 + 32)
convert(::Type{Fahrenheit}, t::Kelvin)   = Fahrenheit(Celsius(t))

convert (generic function with 197 methods)

We can add some nice constructors, so we can initialise each 

In [4]:
Kelvin(t::Temperature) = convert(Kelvin, t)
Celsius(t::Temperature) = convert(Celsius, t)
Fahrenheit(t::Temperature) = convert(Fahrenheit, t)

Fahrenheit

In [5]:
Kelvin(Fahrenheit(23))

Kelvin(268.15)

In [6]:
Fahrenheit(3) + Celsius(4) + Kelvin(4)

MethodError: MethodError: no method matching +(::Fahrenheit, ::Celsius)
Closest candidates are:
  +(::Any, ::Any, !Matched::Any, !Matched::Any...) at operators.jl:502

Adding arithmetic operations for our types by extending the `Base` operators.

In [7]:
import Base: +, -, *  # operators are functions, we can extend them easily!

+(x::Temperature, y::Temperature) = +(promote(x,y)...)
-(x::Temperature, y::Temperature) = -(promote(x,y)...)

+(x::T, y::T) where {T <: Temperature} = T(x.value + y.value)
-(x::T, y::T) where {T <: Temperature} = T(x.value - y.value)

*(x::Number, y::T) where {T <: Temperature} = T(x * y.value);

Let's test it:

In [8]:
Celsius(37) + Kelvin(10)

Kelvin(320.15)

In [9]:
Celsius(Celsius(37) + Kelvin(10))

Celsius(47.0)

In [10]:
Fahrenheit(3) + Celsius(4) + Kelvin(4)

Kelvin(265.0388888888889)

A nice way to create some syntactic sugar:

In [11]:
const °C = Celsius(1)
const °F = Fahrenheit(1)
const K = Kelvin(1);

In [12]:
5°F, 23K, 42°C

(Fahrenheit(5.0), Kelvin(23.0), Celsius(42.0))

In [13]:
42°C + 10K

Kelvin(325.15)

### Alright, let's have a look behind the scenes...

In [14]:
example_calculation() = Fahrenheit(2) + Celsius(3) + Kelvin(4)

example_calculation (generic function with 1 method)

In [15]:
@code_llvm example_calculation()


;  @ In[14]:1 within `example_calculation'
define { double } @julia_example_calculation_13011() {
top:
  ret { double } { double 0x407077BBBBBBBBBB }
}


LLVM figured out that the function returns always the same value... Dooh ;)

This is the "machine code":

In [16]:
@code_native example_calculation()

	.text
; ┌ @ In[14]:1 within `example_calculation'
	movabsq	$140635019387800, %rax  # imm = 0x7FE8245FA398
	vmovsd	(%rax), %xmm0           # xmm0 = mem[0],zero
	retq
	nop
; └


Now a function which is not constant ;)

In [17]:
another_calculation(a, b, c) = Fahrenheit(a) + Celsius(b) + Kelvin(c)

another_calculation (generic function with 1 method)

In [18]:
@code_llvm another_calculation(1, 2, 3)


;  @ In[17]:1 within `another_calculation'
define { double } @julia_another_calculation_13034(i64, i64, i64) {
top:
; ┌ @ In[1]:12 within `Type'
; │┌ @ number.jl:7 within `convert'
; ││┌ @ float.jl:60 within `Type'
     %3 = sitofp i64 %0 to double
; │└└
; │ @ In[1]:4 within `Type'
; │┌ @ number.jl:7 within `convert'
; ││┌ @ float.jl:60 within `Type'
     %4 = sitofp i64 %1 to double
; │└└
; │ @ In[1]:8 within `Type'
; │┌ @ number.jl:7 within `convert'
; ││┌ @ float.jl:60 within `Type'
     %5 = sitofp i64 %2 to double
; └└└
; ┌ @ operators.jl:502 within `+' @ In[7]:3
; │┌ @ promotion.jl:284 within `promote'
; ││┌ @ promotion.jl:261 within `_promote'
; │││┌ @ In[3]:6 within `convert'
; ││││┌ @ promotion.jl:315 within `-' @ float.jl:397
       %6 = fadd double %3, -3.200000e+01
; ││││└
; ││││┌ @ promotion.jl:314 within `*' @ float.jl:399
       %7 = fmul double %6, 5.000000e+00
; ││││└
; ││││┌ @ promotion.jl:316 within `/' @ float.jl:401
       %8 = fdiv double %7, 9.000000e+00
; │└└└└

### After removing the comments, we have basically this LLVM code:

```julia
another_calculation(1, 2, 3)
```

```julia
define { double } @julia_another_calculation_13051(i64, i64, i64) {
top:
     %3 = sitofp i64 %0 to double
     %4 = sitofp i64 %1 to double
     %5 = sitofp i64 %2 to double
       %6 = fadd double %3, -3.200000e+01
       %7 = fmul double %6, 5.000000e+00
       %8 = fdiv double %7, 9.000000e+00
   %9 = fadd double %8, %4
       %10 = fadd double %9, 2.731500e+02
   %11 = fadd double %10, %5
  %.fca.0.insert = insertvalue { double } undef, double %11, 0
  ret { double } %.fca.0.insert
}
```

And the machine code:

In [19]:
@code_native another_calculation(2.0, 3.0, 4.0)

	.text
; ┌ @ In[17]:1 within `another_calculation'
	movabsq	$140635019389544, %rax  # imm = 0x7FE8245FAA68
; │┌ @ operators.jl:502 within `+' @ In[7]:3
; ││┌ @ promotion.jl:284 within `promote'
; │││┌ @ promotion.jl:261 within `_promote'
; ││││┌ @ In[3]:6 within `convert'
; │││││┌ @ promotion.jl:315 within `-' @ float.jl:397
	vaddsd	(%rax), %xmm0, %xmm0
	movabsq	$140635019389552, %rax  # imm = 0x7FE8245FAA70
; ││││└└
; ││││┌ @ float.jl:399 within `convert'
	vmulsd	(%rax), %xmm0, %xmm0
	movabsq	$140635019389560, %rax  # imm = 0x7FE8245FAA78
; ││││└
; ││││┌ @ In[3]:6 within `convert'
; │││││┌ @ promotion.jl:316 within `/' @ float.jl:401
	vdivsd	(%rax), %xmm0, %xmm0
; ││└└└└
; ││ @ operators.jl:502 within `+' @ In[7]:3 @ In[7]:6 @ float.jl:395
	vaddsd	%xmm1, %xmm0, %xmm0
	movabsq	$140635019389568, %rax  # imm = 0x7FE8245FAA80
; ││ @ operators.jl:502 within `+' @ In[7]:3
; ││┌ @ promotion.jl:284 within `promote'
; │││┌ @ promotion.jl:261 within `_promote'
; ││││┌ @ In[3]:3 within `convert'

### The machine code without comments...


```julia
another_calculation(1, 2, 3)
```

```
.text
movabsq	$140703640448904, %rax  # imm = 0x7FF81E81EF88
vaddsd	(%rax), %xmm0, %xmm0
movabsq	$140703640448912, %rax  # imm = 0x7FF81E81EF90
vmulsd	(%rax), %xmm0, %xmm0
movabsq	$140703640448920, %rax  # imm = 0x7FF81E81EF98
vdivsd	(%rax), %xmm0, %xmm0
vaddsd	%xmm1, %xmm0, %xmm0
movabsq	$140703640448928, %rax  # imm = 0x7FF81E81EFA0
vaddsd	(%rax), %xmm0, %xmm0
vaddsd	%xmm2, %xmm0, %xmm0
retq
nopw	%cs:(%rax,%rax)
```

## What about the actual performance?

In [20]:
using BenchmarkTools

In [21]:
@benchmark another_calculation(temperatures...) setup=(temperatures=rand(3))

BenchmarkTools.Trial: 
  memory estimate:  64 bytes
  allocs estimate:  4
  --------------
  minimum time:     60.375 ns (0.00% GC)
  median time:      61.824 ns (0.00% GC)
  mean time:        72.482 ns (10.64% GC)
  maximum time:     42.251 μs (99.73% GC)
  --------------
  samples:          10000
  evals/sample:     981

#### Remember, the same "pure Python" function took about `3.5µs`!

### The power of multiple dispatch

We can easily define our own `rand` method for `Kelvin` temperatures ()

In [22]:
using Random
Random.rand(rng::AbstractRNG, ::Random.SamplerType{Kelvin}) =  Kelvin(rand() * 5000)

In [23]:
rand(Kelvin)

Kelvin(573.4160148870127)

In [24]:
temperatures = rand(Kelvin, 1_000_000)

1000000-element Array{Kelvin,1}:
 Kelvin(4940.805269012501) 
 Kelvin(4159.6252383333585)
 Kelvin(2823.6053707426813)
 Kelvin(2389.4151351240666)
 Kelvin(2841.302997100952) 
 Kelvin(1556.5514289617288)
 Kelvin(4221.379623454689) 
 Kelvin(2450.711459006494) 
 Kelvin(51.52083215364867) 
 Kelvin(2696.1573561675445)
 Kelvin(3432.294371022457) 
 Kelvin(3800.8990866006907)
 Kelvin(1597.2238000596005)
 ⋮                         
 Kelvin(4985.844991792913) 
 Kelvin(4920.9558943487355)
 Kelvin(2105.125091152399) 
 Kelvin(4318.632039422512) 
 Kelvin(1427.58709930353)  
 Kelvin(197.87771654400154)
 Kelvin(1859.4085308396357)
 Kelvin(3449.391669134375) 
 Kelvin(468.06190048947036)
 Kelvin(282.1718686666497) 
 Kelvin(3639.4564687634524)
 Kelvin(1702.5349935977374)

### Is this fast?

In [25]:
@benchmark rand(Kelvin, 1_000_000)

BenchmarkTools.Trial: 
  memory estimate:  7.63 MiB
  allocs estimate:  2
  --------------
  minimum time:     1.683 ms (0.00% GC)
  median time:      2.207 ms (0.00% GC)
  mean time:        2.154 ms (10.81% GC)
  maximum time:     50.696 ms (95.38% GC)
  --------------
  samples:          2317
  evals/sample:     1

### Yep... 2ms, 2 allocs, 8 MiB memory

### Convert them to `Celsius` by using our type constructor and the `.`-notation for element-wise operation (similar to Matlab or `ufuncs` in numpy)

In [26]:
Celsius.(temperatures)

1000000-element Array{Celsius,1}:
 Celsius(4667.655269012502)  
 Celsius(3886.4752383333584) 
 Celsius(2550.455370742681)  
 Celsius(2116.2651351240665) 
 Celsius(2568.152997100952)  
 Celsius(1283.401428961729)  
 Celsius(3948.229623454689)  
 Celsius(2177.561459006494)  
 Celsius(-221.62916784635132)
 Celsius(2423.0073561675445) 
 Celsius(3159.144371022457)  
 Celsius(3527.7490866006906) 
 Celsius(1324.0738000596007) 
 ⋮                           
 Celsius(4712.694991792913)  
 Celsius(4647.805894348736)  
 Celsius(1831.9750911523988) 
 Celsius(4045.482039422512)  
 Celsius(1154.43709930353)   
 Celsius(-75.27228345599843) 
 Celsius(1586.2585308396356) 
 Celsius(3176.241669134375)  
 Celsius(194.91190048947038) 
 Celsius(9.02186866664971)   
 Celsius(3366.3064687634524) 
 Celsius(1429.3849935977373) 

### Is this fast?

In [27]:
@benchmark Celsius.($temperatures)

BenchmarkTools.Trial: 
  memory estimate:  7.63 MiB
  allocs estimate:  2
  --------------
  minimum time:     1.322 ms (0.00% GC)
  median time:      1.568 ms (0.00% GC)
  mean time:        1.754 ms (14.68% GC)
  maximum time:     49.637 ms (96.85% GC)
  --------------
  samples:          2840
  evals/sample:     1

### Yep... 1.5ms, 2 allocs, 8 MiB memory

## And we magically get all the other methods of the `rand()` function:

In [28]:
rand(Kelvin, 100, 200, 300)

100×200×300 Array{Kelvin,3}:
[:, :, 1] =
 Kelvin(3906.61)  Kelvin(4437.37)  …  Kelvin(3875.14)  Kelvin(1893.55)
 Kelvin(3054.67)  Kelvin(4115.37)     Kelvin(4999.88)  Kelvin(143.544)
 Kelvin(1380.18)  Kelvin(3493.79)     Kelvin(2188.36)  Kelvin(4648.81)
 Kelvin(4115.61)  Kelvin(3062.11)     Kelvin(4697.77)  Kelvin(945.323)
 Kelvin(318.606)  Kelvin(1429.2)      Kelvin(66.93)    Kelvin(578.634)
 Kelvin(3611.88)  Kelvin(2579.11)  …  Kelvin(3869.74)  Kelvin(4388.94)
 Kelvin(3682.26)  Kelvin(1389.7)      Kelvin(4817.19)  Kelvin(4577.52)
 Kelvin(1407.09)  Kelvin(3065.22)     Kelvin(984.148)  Kelvin(2257.02)
 Kelvin(1880.35)  Kelvin(2252.3)      Kelvin(4028.52)  Kelvin(1843.76)
 Kelvin(2292.25)  Kelvin(1580.69)     Kelvin(2401.24)  Kelvin(601.959)
 Kelvin(4912.33)  Kelvin(2622.9)   …  Kelvin(3283.0)   Kelvin(2790.19)
 Kelvin(2838.59)  Kelvin(557.1)       Kelvin(3490.04)  Kelvin(4286.85)
 Kelvin(4887.14)  Kelvin(3111.83)     Kelvin(1482.61)  Kelvin(471.215)
 ⋮                                 ⋱